In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import os

# -------------------------
# 🔹 Load Tiny ImageNet Data using TensorFlow
# -------------------------
def load_data():
    # Directories for Tiny ImageNet
    train_dir = 'tiny-imagenet-200/train/'
    val_dir = 'tiny-imagenet-200/val/'
    
    # Load the training images and labels
    x_train, y_train = [], []
    class_names = os.listdir(train_dir)
    class_names.sort()  # Sorting to ensure class order remains the same
    
    for i, class_name in enumerate(class_names):
        class_dir = os.path.join(train_dir, class_name, 'images')
        for image_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, image_name)
            img = tf.io.read_file(img_path)
            img = tf.image.decode_jpeg(img, channels=3)  # Decode the image
            img = tf.image.resize(img, [64, 64])  # Resize to 64x64
            img = img / 255.0  # Normalize the image
            x_train.append(img)
            y_train.append(i)
    
    x_train = np.array(x_train, dtype=np.float32)
    y_train = np.array(y_train)
    y_train = to_categorical(y_train, 200)  # One-hot encode labels

    # Load the validation images and labels
    x_val, y_val = [], []
    val_images = os.listdir(val_dir)
    with open(os.path.join(val_dir, 'val_annotations.txt'), 'r') as f:
        val_info = f.readlines()
    
    for line in val_info:
        image_name, class_name = line.strip().split('\t')[:2]
        img_path = os.path.join(val_dir, 'images', image_name)
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)  # Decode the image
        img = tf.image.resize(img, [64, 64])  # Resize to 64x64
        img = img / 255.0  # Normalize the image
        x_val.append(img)
        y_val.append(class_names.index(class_name))
    
    x_val = np.array(x_val, dtype=np.float32)
    y_val = np.array(y_val)
    y_val = to_categorical(y_val, 200)  # One-hot encode labels

    return x_train, y_train, x_val, y_val, 200

# -------------------------
# 🔹 Build Model (ResNet50)
# -------------------------
def build_model(num_classes):
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(64, 64, 3))

    # Custom layers
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation="relu")(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model

# -------------------------
# 🔹 Train Model
# -------------------------
def train_and_evaluate():
    x_train, y_train, x_val, y_val, num_classes = load_data()
    model = build_model(num_classes)

    print("\n--- Training on Tiny ImageNet ---")
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=20, batch_size=64, verbose=1)

    print("\n--- Evaluating on Full Dataset ---")
    y_pred = np.argmax(model.predict(x_val), axis=1)
    y_true = np.argmax(y_val, axis=1)
    
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    return model, x_val, y_pred, y_true, cm

# -------------------------
# 🔹 Run Training & Evaluation
# -------------------------
trained_model, val_images, val_preds, val_labels, final_cm = train_and_evaluate()

# -------------------------
# 🔹 Plot Normalized Confusion Matrix for Tiny ImageNet
# -------------------------
def plot_confusion_matrix(cm, labels):
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm_normalized, annot=False, fmt=".2f", cmap="Blues", xticklabels=labels, yticklabels=labels)
    plt.title("Final Normalized Confusion Matrix - Tiny ImageNet")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()

# Labels for Tiny ImageNet classes
tiny_imagenet_labels = list(range(200))  # 200 classes
plot_confusion_matrix(final_cm, tiny_imagenet_labels)


2025-03-25 21:08:12.094800: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-25 21:08:12.105954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742951292.118803 3160641 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742951292.122652 3160641 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-25 21:08:12.135767: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

ResourceExhaustedError: {{function_node __wrapped__RealDiv_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:RealDiv] name: 

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import os

# -------------------------
# 🔹 Replay Buffer for Balancing Data
# -------------------------
class ReplayBuffer:
    def __init__(self, capacity=5000):
        self.buffer = []
        self.capacity = capacity

    def add_samples(self, x, y):
        if len(self.buffer) >= self.capacity:
            self.buffer = self.buffer[len(y):]  # Remove old samples
        self.buffer.extend(zip(x, y))

    def get_samples(self, batch_size=1000):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        samples = [self.buffer[i] for i in indices]
        x_replay, y_replay = zip(*samples)
        return np.array(x_replay), np.array(y_replay)

# -------------------------
# 🔹 Load Tiny ImageNet Data using TensorFlow
# -------------------------
def load_data():
    # Directories for Tiny ImageNet
    train_dir = 'tiny-imagenet-200/train/'
    val_dir = 'tiny-imagenet-200/val/'
    
    # Load the training images and labels
    x_train, y_train = [], []
    class_names = os.listdir(train_dir)
    class_names.sort()  # Sorting to ensure class order remains the same
    
    for i, class_name in enumerate(class_names):
        class_dir = os.path.join(train_dir, class_name, 'images')
        for image_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, image_name)
            img = tf.io.read_file(img_path)
            img = tf.image.decode_jpeg(img, channels=3)  # Decode the image
            img = tf.image.resize(img, [64, 64])  # Resize to 64x64
            img = img / 255.0  # Normalize the image
            x_train.append(img)
            y_train.append(i)
    
    x_train = np.array(x_train, dtype=np.float32)
    y_train = np.array(y_train)
    y_train = to_categorical(y_train, 200)  # One-hot encode labels

    # Load the validation images and labels
    x_val, y_val = [], []
    val_images = os.listdir(val_dir)
    with open(os.path.join(val_dir, 'val_annotations.txt'), 'r') as f:
        val_info = f.readlines()
    
    for line in val_info:
        image_name, class_name = line.strip().split('\t')[:2]
        img_path = os.path.join(val_dir, 'images', image_name)
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)  # Decode the image
        img = tf.image.resize(img, [64, 64])  # Resize to 64x64
        img = img / 255.0  # Normalize the image
        x_val.append(img)
        y_val.append(class_names.index(class_name))
    
    x_val = np.array(x_val, dtype=np.float32)
    y_val = np.array(y_val)
    y_val = to_categorical(y_val, 200)  # One-hot encode labels

    return x_train, y_train, x_val, y_val, 200

# -------------------------
# 🔹 Improve Model Fine-Tuning (Replay Buffer & Knowledge Distillation)
# -------------------------
def fine_tune_model(trained_model, x_train, y_train, x_test, y_test):
    # Learning rate decay
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-4,
        decay_steps=20000,
        decay_rate=0.9
    )
    optimizer = Adam(learning_rate=lr_schedule)

    trained_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    print("\n--- Fine-Tuning Model on Tiny ImageNet ---")
    history_finetune = trained_model.fit(x_train, y_train, validation_data=(x_test, y_test),
                                         epochs=50, batch_size=64, verbose=1)
    return history_finetune

# -------------------------
# 🔹 Final Training Setup
# -------------------------
def train_final_model():
    x_train, y_train, x_test, y_test, num_classes = load_data()

    # Initialize ResNet model
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(64, 64, 3))
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation="relu")(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

    # Train the final model
    print("\n--- Initial Training on Tiny ImageNet ---")
    history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50, batch_size=64, verbose=1)

    return model, x_test, y_test

# -------------------------
# 🔹 Train and Evaluate Final Model
# -------------------------
trained_model, final_x_test, final_y_test = train_final_model()

# Fine-tune the model further
fine_tune_model(trained_model, final_x_test, final_y_test, final_x_test, final_y_test)

# Compute final confusion matrix
final_predictions = np.argmax(trained_model.predict(final_x_test), axis=1)
final_true_labels = np.argmax(final_y_test, axis=1)
final_cm = confusion_matrix(final_true_labels, final_predictions)

# -------------------------
# 🔹 Plot Normalized Confusion Matrix
# -------------------------
def plot_confusion_matrix(cm, labels):
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm_normalized, annot=False, fmt=".2f", cmap="Blues", xticklabels=labels, yticklabels=labels)
    plt.title("Final Normalized Confusion Matrix (Tiny ImageNet)")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()

plot_confusion_matrix(final_cm, list(range(200)))  # 200 classes for Tiny ImageNet


In [ ]:
import os
import random
import matplotlib.pyplot as plt

# -------------------------
# 🔹 Save the Trained Model
# -------------------------
model_filename = "tiny_imagenet_resnet50_model.h5"
trained_model.save(model_filename)
print(f"Model saved as {model_filename}")

# -------------------------
# 🔹 Load Tiny ImageNet Labels (Human Readable)
# -------------------------
tiny_imagenet_label_names = [
    'n02118413_bangalore_terrier', 'n02119789_kit_fox', 'n02120505_fox_squirrel', 'n02127052_lynx', 'n02128385_leopard',
    'n02129165_snow_leopard', 'n02129604_tiger_cat', 'n02130308_cheetah', 'n02132136_black_footed_ferret', 'n02134084_ice_fish',
    # ... you need to include all 200 class names here
    # Add the rest of the 200 class names from Tiny ImageNet
]

# -------------------------
# 🔹 Display 50 Random Predictions with True Labels
# -------------------------
num_samples = 50
indices = random.sample(range(len(test_images)), num_samples)

plt.figure(figsize=(15, 15))
for i, idx in enumerate(indices):
    image = test_images[idx]
    true_label = tiny_imagenet_label_names[test_labels[idx]]
    predicted_label = tiny_imagenet_label_names[test_preds[idx]]

    plt.subplot(10, 5, i + 1)
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"True: {true_label}\nPred: {predicted_label}")

plt.tight_layout()
plt.show()
